Import the libraries

In [98]:
import json
import sqlite3
import pandas as pd
from langchain_openai import ChatOpenAI
import os
import re

Connect to sqlite db

In [ ]:
cols = ['showname','first_airing','imdb','lang','description']
def fetch_rows(limit: int, offset: int):
    conn = sqlite3.connect("movie_db.sqlite")
    cursor = conn.cursor()
    executor = cursor.execute(f'select {",".join(cols)} from tvmaze order by showname asc limit {limit} offset {offset}')
    data = executor.fetchall()
    cursor.close()
    conn.close()
    return data

pd.DataFrame(fetch_rows(100, 0), columns=cols)

,showname,first_airing,imdb,lang,description
0,"""How Ya Been?"" with Jules LeBlanc",2021-05-22,None,English,<p>Jules LeBlanc catches up with her former Ch...
1,"""Алиби"" на двоих",2011-08-13,None,Russian,<p>Hapless swindler Gosha steals Guarneri's vi...
2,"""Везёт""",2021-11-04,None,Russian,"<p>Ivan Potapov, a former speedway champion, m..."
3,"""Кедр"" пронзает небо",2011-04-11,tt3189080,Russian,<p>Year 1940. The young scientist Sergei Lykov...
4,"""Лучшая"" неделя моей жизни",2016-10-10,None,Russian,"<p>Comedy series <b>""Лучшая"" неделя моей жизни..."
...,...,...,...,...,...
95,1 Contra Todos,2016-06-20,tt5822138,Portuguese,"<p>The story of Cadu, a public defender and la..."
96,1 Day With...,2004-09-04,tt0426635,English,<p>Soap actor Wally Kurth (General Hospital) f...
97,1 For All,2019-10-15,tt11580350,English,<p>An original comedy sketch web series by Dee...
98,1 Girl 5 Gays,2009-10-14,tt1606854,English,<p><b>1 Girl 5 Gays</b> was a Canadian talk sh...


Prepare NLU

In [ ]:
# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="deepseek-r1-distill-llama-8b",
    openai_api_key="fake",
    openai_api_base='http://127.0.0.1:1234/v1',
)


Tools for extracting metadata

In [114]:
def generate_prompt(values: list[str]):
    with open(f'./prompts/movie_metadata_extraction.txt', 'r') as file:
        data = file.read().rstrip()
        index=0
            
        while(data.find(f"!<INPUT_{index}>") > 0):
            data = data.replace(f"!<INPUT_{index}>", values[index] if len(values) > index is not None else '')
            index+=1
        return data

    return None;

Define job to process data

In [ ]:
from langchain.schema import BaseMessage, HumanMessage

def process_job(limit: int, offset: int) -> list[dict]:
    results = []
    data = fetch_rows(limit, offset)
    for row in data:
        if(row[4] is not None and len(row[4]) > 0):
            prompt = generate_prompt([row[4]])
            output = llm.invoke([HumanMessage(content=prompt)])
            json_str = re.findall(r"```json(.*?)```", output.text(),re.DOTALL)
            if(len(json_str) > 0):
                try:
                    conn = sqlite3.connect("movie_db.sqlite")
                    cursor = conn.cursor()
                    cursor.execute(f'update tvmaze set metadata = \'{json.dumps(results[0])}\' where showname = \'{row[0]}\'')
                    conn.commit()
                    cursor.close()
                    conn.close()
                    results.append(json.loads(json_str[0]))
                except:
                    pass
            
    return results;

Process the data

In [116]:
results = process_job(1,1)

In [141]:
conn = sqlite3.connect("movie_db.sqlite")
cursor = conn.cursor()
executor = cursor.execute(f'update tvmaze set metadata = \'{json.dumps(results[0])}\' where showname = \'"Алиби" на двоих\'')
conn.commit()
cursor.close()
conn.close()